In [1]:
import datetime as dt
import sqlalchemy as sa
import os
import sys
import pandas as pd

In [2]:
username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions/'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, clear_table, load_key, decrypt

key1 = 'user_key.ky'
key_file1 = 'user_key.txt'
key2 = 'pass_key.ky'
key_file2 = 'pass_key.txt'

key1 = load_key(f_path, key1)
uid = decrypt(f_path, key_file1, key1)

key2 = load_key(f_path, key2)
passwd = decrypt(f_path, key_file2, key2)

# Setup connection parameters
server = 'danvuk.database.windows.net'
dbase = 'Financial_Securities'

# Create a connection to the database
s, e = create_connection(server, dbase, uid, passwd)
s1 = s()  # Instantiate a session object

In [3]:
Base = sa.orm.declarative_base()

class Data_STG(Base):
    
    """
    SQLAlchemy ORM class representing the 'Data_STG' table in the 'Equities' schema.

    Attributes:
        __tablename__ (str): The name of the table in the database.
        __table_args__ (dict): Additional arguments for the table, including schema name.
        Date (Column): The date column, part of the composite primary key.
        Description (Column): A description column, part of the composite primary key.
        Description2 (Column): An additional description column.
        Int_Value1 (Column): An integer column.
    """
    
    __tablename__ = 'Data_STG'
    __table_args__ = {'schema': 'Equities'}
    Date = sa.Column(sa.Date, primary_key=True)
    Description = sa.Column(sa.String, primary_key=True)
    Description2 = sa.Column(sa.String)
    Int_Value1 = sa.Column(sa.Integer)

In [4]:
# Clear the existing data in the Data_STG table
clear_table(s1, 'Financial_Securities.Equities.Data_STG')

In [5]:
in_file1 = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Data-Source-Files/SP500_GICS_Combined.csv'  # Path to 1st CSV file
in_file2 = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Data-Source-Files/SP500_Equities_Prices.csv'  # Path to 2nd CSV file

curr_date = dt.datetime.now()  # Current date and time

# Valisdate if both files exist
if (os.path.isfile(in_file1)) and (os.path.isfile(in_file2)):
    
    # Load the 1st CSV file into 1st DataFrame
    df_equities1 = pd.read_csv(in_file1)
    
    # Load the 2nd CSV file into 2nd DataFrame
    df_equities2 = pd.read_csv(in_file2)
    
    # Drop the last row of comment data coming from Barchart file
    df_equities2.drop(df_equities2.tail(1).index, inplace=True)
    
    # Check if values in df_equities1['Ticker'] exist in df_equities2['Symbol']
    matches1 = df_equities1['Ticker'].isin(df_equities2['Symbol'])
    all_match1 = matches1.all()
    
    # Check if values in df_equities2['Symbol'] exist in df_equities1['Ticker']
    matches2 = df_equities2['Symbol'].isin(df_equities1['Ticker'])
    all_match2 = matches2.all()

    # if tickers match in both files, the latest S&P 500 ticker list is valid
    if all_match1 and all_match2:
                      
        # Select relevant columns
        df_equities1 = df_equities1[['Ticker', 'Name', 'Sub_Industry_ID']]
    
        # Iterate through the rows of the DataFrame
        for index, row in df_equities1.iterrows():
            try:
                # Create a new Data_STG instance for each row
                q1 = Data_STG(
                    Date=curr_date,
                    Description=row['Ticker'],
                    Description2=row['Name'],
                    Int_Value1=row['Sub_Industry_ID']
                )
                s1.add(q1)  # Add the instance to the session
        
            # Handle SQLAlchemy errors if they occur during adding the object
            except sa.exc.SQLAlchemyError as e:
                message = f"Issue with updating Data_STG database table for Date: {curr_date} and Ticker: {row['Ticker']}. Error: {e}"
                print(message)
    
        # Commit all changes to the database
        s1.commit()
    
        print("Database data load is complete")
    
    
        # SQL query to count the number of records in the Data_STG table
        sql_stat = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""

        try: 
            result = e.execute(sql_stat)  # Execute the count query
            cnt_recs = result.scalar()  # Get the count of records
    
         # Handle SQLAlchemy errors if they occur during query execution
        except sa.exc.SQLAlchemyError as e:
            # Print the error
            print(f"Issue querying Data_STG database table for count! Error: {e}")

        
        # Compare the record counts and print the result    
        if cnt_recs < len(df_equities1):
            print(f"Only {cnt_recs} records out of {len(df_equities1)} records were loaded into the Data_STG table!")
        else:
            print(f"All {cnt_recs} records were loaded into the Data_STG table!")
            
    else:
        print(f"Tickers in the {os.path.basename(in_file1)} file don't match the tickers in the {os.path.basename(in_file1)} file")

else:
    print("File not found!")  # Print a message if the file does not exist

Database data load is complete
All 503 records were loaded into the Data_STG table!


In [6]:
s1.close()  # Close the session